### 3.2.1.Интеграция SQLAlchemy с Flask.

Flask не имеет встроенного ORM.  
SQLAlchemy — самая популярная ORM для Python. Позволяет работать с БД через Python-объекты. Автоматизирует создание/изменение таблиц. Обеспечивает безопасность (защита от SQL-инъекций). Поддерживает PostgreSQL, MySQL, SQLite и другие СУБД. Flask-SQLAlchemy — официальное расширение Flask для работы с SQLAlchemy. Alembic (опционально) — для миграций базы данных.

**Модель** — Python-класс, который представляет таблицу в БД  
**Сессия** — область взаимодействия с БД (аналог транзакции)  
**Запрос** (Query) — объект для выборки данных

Структура проекта

[myblog/](https://github.com/AnatolyKuzmin/Python/blob/main/3_Веб-разработка_(Flask_Django)/myblog_SQLAlchemy)  
├── app.py          # Основное приложение  
├── models.py       # Модели данных  
├── config.py       # Конфигурация  
└── templates/      # Шаблоны (если нужно)  

Примеры:  
Создание записи  
```
from models import User, Post
from app import create_app

app = create_app()

with app.app_context():
    # Создаём пользователя
    new_user = User(username='john_doe', email='john@example.com')
    db.session.add(new_user)
    
    # Создаём пост
    new_post = Post(title='First Post', content='Hello World!', author=new_user)
    db.session.add(new_post)
    
    # Фиксируем изменения
    db.session.commit()
```
Чтение данных  
```
with app.app_context():
    # Получаем всех пользователей
    users = User.query.all()
    
    # Получаем пользователя по username
    user = User.query.filter_by(username='john_doe').first()
    
    # Получаем все посты пользователя
    posts = Post.query.filter_by(user_id=user.id).all()
```
Обновление записи  
```
with app.app_context():
    user = User.query.get(1)  # Получаем пользователя с id=1
    user.email = 'new_email@example.com'
    db.session.commit()
```
Удаление записи  
```
with app.app_context():
    post = Post.query.get(1)  # Получаем пост с id=1
    db.session.delete(post)
    db.session.commit()
```

!!! Важные замечания:  
- Все изменения должны завершаться db.session.commit()  
- При ошибках используйте db.session.rollback()  
- Все операции с БД должны выполняться в контексте приложения  
- Используйте with app.app_context(): если вне route-функций

### 3.2.2.Создание моделей и миграции.

Для чего нужная миграция? При изменении моделей SQLAlchemy не обновляет автоматически структуру БД.  
Значит решение. Alembic — система миграций, которая: Отслеживает изменения моделей; Генерирует SQL-скрипты для обновления БД; Позволяет откатывать изменения. 

Миграция — файл с SQL-инструкциями для изменения структуры БД
Ревизия (Revision) — версия миграции
Upgrade/Downgrade — применение/откат изменений

Установка пакетов `pip install alembic`  
Инициализация миграций `flask db init`  
Создаст папку migrations с структурой  
migrations/  
├── versions/       # Здесь будут храниться файлы миграций  
├── env.py          # Конфигурация Alembic  
└── script.py.mako  # Шаблон для миграций  
Важно проверить настройки в migrations/env.py  
```
from models import db
target_metadata = db.metadata  # Указываем метаданные моделей
```

Создание моделей  
```
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class User(db.Model):
    __tablename__ = 'users'
    
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, server_default=db.func.now())
    
    posts = db.relationship('Post', backref='author', lazy=True)

class Post(db.Model):
    __tablename__ = 'posts'
    
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'))
```

Создание первой миграции  
Генерация миграции  
```
flask db migrate -m "Initial migration"
```
Пример сгенерированной миграции  
```
def upgrade():
    op.create_table('users',
        sa.Column('id', sa.Integer(), nullable=False),
        sa.Column('username', sa.String(length=80), nullable=False),
        ...
    )
    op.create_table('posts',
        ...
    )

def downgrade():
    op.drop_table('posts')
    op.drop_table('users')
```
Применение миграции  
```
flask db upgrade
```

Изменение моделей и создание новых миграций   
Добавим новое поле в User  
```
class User(db.Model):
    # ... предыдущие поля ...
    is_active = db.Column(db.Boolean, default=True)  # Новое поле
```
Создаём миграцию  
```
flask db migrate -m "Add is_active to User"
```
Проверяем сгенерированный SQL  
```
def upgrade():
    op.add_column('users', sa.Column('is_active', sa.Boolean(), nullable=True))
    op.execute("UPDATE users SET is_active = True")  # Заполняем существующие записи
    op.alter_column('users', 'is_active', nullable=False)

def downgrade():
    op.drop_column('users', 'is_active')
```
Применяем изменения  
```
flask db upgrade
```




### 3.2.3.CRUD операции.

### 3.2.4.Практика: расширение блога с базой данных.